In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from __future__ import division
%matplotlib inline

In [2]:
X = pd.DataFrame.from_csv("X_data.csv")
y = pd.DataFrame.from_csv("y_data.csv")
y = np.ravel(y)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [4]:
# Train neural network
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train.iloc[:,1:], y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [13]:
y_test_predict = pd.DataFrame(clf.predict_proba(X_test.iloc[:,1:]), index=X_test.index)

In [14]:
grades = [grade for grade in X_test.columns if grade[:6] == "grade_"]

results = {}

for grade in grades:
    good = y_test_predict[y_test == False][X_test[grade] == True][0].mean()
    bad = y_test_predict[y_test == True][X_test[grade] == True][0].mean()
    results[grade[-1]] = {'Good': good, "Bad": bad}

print pd.DataFrame(results)

/usr/local/lib/python2.7/site-packages/ipykernel-4.2.2-py2.7.egg/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel-4.2.2-py2.7.egg/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


             A         B         C         D         E         F         G
Bad   0.917442  0.841667  0.794434  0.738691  0.715175  0.656285  0.659189
Good  0.935606  0.855116  0.808373  0.754008  0.723817  0.680087  0.641830


Since the probability of repayment for any given loan is high, out model will tend to predict that a loan will get repaid. However, the actual probability of repayment varies. From the above, the model shows potential as, even when broken out by grade, the model shows some predictive power. The only grade this did not hold for is grade G, which may be due to the small size.

Since the model will tend to predict repayment for every loan, the score function is not very useful. Another way to score how well we're doing is to use the model to decide what loans to buy to maximize the loss-adjusted return

## Testing number of hidden layers and neurons